In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# CSV 파일 읽기
source = pd.read_csv('업체정보.csv')
keywords = source['업체명'].tolist()

# 데이터를 저장할 리스트 초기화
results = []

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# 네이버 지도 페이지 열기
driver.get('https://map.naver.com/')

# 검색창 요소 찾기
search = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".input_search"))
)

for keyword in keywords:
    try:
        # 검색창 초기화 (이전 검색어 삭제)
        search.clear()

        # 검색어 입력 및 검색
        search.send_keys(keyword)
        search.send_keys(Keys.ENTER)

        # 프레임 전환 대기 및 프레임 전환 (entryIframe)
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#entryIframe"))
        )

        # '정보 더보기' 버튼 클릭 대기 및 클릭
        more_info_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.fvwqf"))
        )
        more_info_button.click()

        # 정보 텍스트 추출
        description = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.T8RFa.CEyr5"))
        )
        info_text = description.text
        
        # 결과를 리스트에 추가
        results.append({'업체명': keyword, '업체정보': info_text})

        # 검색창 기본 프레임으로 전환
        driver.switch_to.default_content()

    except Exception as e:
        print(f"에러 발생: {str(e)}")

# 리스트를 데이터프레임으로 변환
df_result = pd.DataFrame(results)

# 데이터프레임 출력
print(df_result)

# 브라우저 닫기
driver.quit()


에러 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x0082C1C3+27395]
	(No symbol) [0x007C3DC4]
	(No symbol) [0x006C1B7F]
	(No symbol) [0x0069E483]
	(No symbol) [0x0072A06F]
	(No symbol) [0x0073C3D6]
	(No symbol) [0x00723736]
	(No symbol) [0x006F7541]
	(No symbol) [0x006F80BD]
	GetHandleVerifier [0x00AE3A93+2876371]
	GetHandleVerifier [0x00B37F5D+3221661]
	GetHandleVerifier [0x008AD634+556916]
	GetHandleVerifier [0x008B474C+585868]
	(No symbol) [0x007CCE04]
	(No symbol) [0x007C9818]
	(No symbol) [0x007C99B7]
	(No symbol) [0x007BBF0E]
	BaseThreadInitThunk [0x76F5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778180CE+286]
	RtlGetAppContainerNamedObjectPath [0x7781809E+238]

에러 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x0082C1C3+27395]
	(No sym

In [18]:
df_result.to_excel('업체정보_리뷰.xlsx', index=False, engine='openpyxl')

In [20]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# CSV 파일 읽기
source = pd.read_csv('업체정보.csv')
keywords = source['업체명'].tolist()

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# 네이버 지도 페이지 열기
driver.get('https://map.naver.com/')

# 결과를 저장할 리스트 생성
results = []

for keyword in keywords:
    try:
        # 검색창 대기 및 입력
        search = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".input_search"))
        )

        # 검색창 초기화 (이전 검색어 삭제)
        search.clear()

        # 검색어 입력 및 검색
        search.send_keys(keyword)
        search.send_keys(Keys.ENTER)

        # 프레임 전환 대기 및 프레임 전환 (entryIframe)
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#entryIframe"))
        )

        # 리뷰 탭 클릭 대기 및 클릭
        review_tab = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a.tpj9w._tab-menu[role='tab'][aria-selected='false']"))
        )
        review_tab.click()

        # 4번째 탭 선택
        target_tab = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a.tpj9w._tab-menu[role='tab'][aria-selected='false']:nth-of-type(4)"))
        )
        target_tab.click()

        # 스크롤을 내리면서 모든 리뷰 가져오기
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            try:
                more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.fvwqf"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", more_button)
                more_button.click()
                time.sleep(2)  # 클릭 후 대기 시간

                # 스크롤 다운
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            except Exception as e:
                print(f"더보기 버튼이 더 이상 없음: {str(e)}")
                break

        # 모든 리뷰 내용과 방문일 추출
        reviews = driver.find_elements(By.CSS_SELECTOR, "span.zPfVt")
        visit_dates = driver.find_elements(By.CSS_SELECTOR, "span.place_blind")

        review_texts = []
        visit_dates_texts = []

        for review, visit_date in zip(reviews, visit_dates):
            review_text = review.text
            visit_date_text = visit_date.text
            review_texts.append(review_text)
            visit_dates_texts.append(visit_date_text)

        # 결과를 리스트에 추가
        results.extend(list(zip([keyword]*len(reviews), review_texts, visit_dates_texts)))

        # 검색창 기본 프레임으로 전환
        driver.switch_to.default_content()

    except Exception as e:
        print(f"에러 발생: {str(e)}")
    
    finally:
        # 검색창 초기화 (다음 가게로 넘어가기 전에)
        search.clear()

# 데이터프레임 생성
df_result = pd.DataFrame(results, columns=['업체명', '리뷰', '방문일'])

# 데이터프레임 출력
print(df_result)

# 브라우저 닫기
driver.quit()


더보기 버튼이 더 이상 없음: Message: 
Stacktrace:
	GetHandleVerifier [0x0082C1C3+27395]
	(No symbol) [0x007C3DC4]
	(No symbol) [0x006C1B7F]
	(No symbol) [0x00702C65]
	(No symbol) [0x00702D3B]
	(No symbol) [0x0073EC82]
	(No symbol) [0x007239E4]
	(No symbol) [0x0073CB24]
	(No symbol) [0x00723736]
	(No symbol) [0x006F7541]
	(No symbol) [0x006F80BD]
	GetHandleVerifier [0x00AE3A93+2876371]
	GetHandleVerifier [0x00B37F5D+3221661]
	GetHandleVerifier [0x008AD634+556916]
	GetHandleVerifier [0x008B474C+585868]
	(No symbol) [0x007CCE04]
	(No symbol) [0x007C9818]
	(No symbol) [0x007C99B7]
	(No symbol) [0x007BBF0E]
	BaseThreadInitThunk [0x76F5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778180CE+286]
	RtlGetAppContainerNamedObjectPath [0x7781809E+238]

에러 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0082C1C3+27395]
	(No symbol) [0x007C3DC4]
	(No symbol) [0x006C1B7F]
	(No symbol) [0x00702C65]
	(No symbol) [0x00702D3B]
	(No symbol) [0x0073EC82]
	(No symbol) [0x007239E4]
	(No symbol) [0x0073CB24]
	(No symbo

NoSuchElementException: Message: no such element: element not found
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0082C1C3+27395]
	(No symbol) [0x007C3DC4]
	(No symbol) [0x006C1B7F]
	(No symbol) [0x006CFE47]
	(No symbol) [0x006D063E]
	(No symbol) [0x006C774B]
	(No symbol) [0x006C5A84]
	(No symbol) [0x006C88F8]
	(No symbol) [0x006C8970]
	(No symbol) [0x00704652]
	(No symbol) [0x006FA802]
	(No symbol) [0x0072399C]
	(No symbol) [0x006F8C15]
	(No symbol) [0x00723C34]
	(No symbol) [0x0073CB24]
	(No symbol) [0x00723736]
	(No symbol) [0x006F7541]
	(No symbol) [0x006F80BD]
	GetHandleVerifier [0x00AE3A93+2876371]
	GetHandleVerifier [0x00B37F5D+3221661]
	GetHandleVerifier [0x008AD634+556916]
	GetHandleVerifier [0x008B474C+585868]
	(No symbol) [0x007CCE04]
	(No symbol) [0x007C9818]
	(No symbol) [0x007C99B7]
	(No symbol) [0x00

In [23]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# CSV 파일 읽기
source = pd.read_csv('업체정보.csv')

# 업체명과 지역을 결합하여 검색어 리스트 생성
keywords = [f"{row['업체명']} {row['지역']}" for idx, row in source.iterrows()]

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# 네이버 지도 페이지 열기
driver.get('https://map.naver.com/')

# 결과를 저장할 리스트 생성
results = []

for keyword in keywords:
    try:
        # 검색창 대기 및 입력
        search = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".input_search"))
        )

        # 검색창 초기화 (이전 검색어 삭제)
        search.clear()

        # 검색어 입력 및 검색
        search.send_keys(keyword)
        search.send_keys(Keys.ENTER)

        # 프레임 전환 대기 및 프레임 전환 (entryIframe)
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#entryIframe"))
        )

        # 리뷰 탭 클릭 대기 및 클릭
        try:
            review_tab = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "a.tpj9w._tab-menu[role='tab'][aria-selected='false']"))
            )
            review_tab.click()

            # 4번째 탭 선택
            try:
                target_tab = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, "a.tpj9w._tab-menu[role='tab'][aria-selected='false']:nth-of-type(4)"))
                )
                target_tab.click()

                # 최대 100개의 리뷰 수집
                reviews_collected = 0
                while True:
                    try:
                        more_button = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.fvwqf"))
                        )
                        driver.execute_script("arguments[0].scrollIntoView();", more_button)
                        more_button.click()
                        time.sleep(2)  # 클릭 후 대기 시간

                        # 스크롤 다운
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                        # 새로 수집한 리뷰 개수 확인
                        new_reviews = len(driver.find_elements(By.CSS_SELECTOR, "span.zPfVt"))
                        reviews_collected += new_reviews

                        # 최대 100개까지만 수집
                        if reviews_collected >= 100:
                            break

                    except Exception as e:
                        print(f"더보기 버튼이 더 이상 없음: {str(e)}")
                        break

                # 모든 리뷰 내용과 방문일 추출
                reviews = driver.find_elements(By.CSS_SELECTOR, "span.zPfVt")
                visit_dates = driver.find_elements(By.CSS_SELECTOR, "span.place_blind")

                review_texts = []
                visit_dates_texts = []

                for review, visit_date in zip(reviews, visit_dates):
                    review_text = review.text
                    visit_date_text = visit_date.text
                    review_texts.append(review_text)
                    visit_dates_texts.append(visit_date_text)

                # 결과를 리스트에 추가
                results.extend(list(zip([keyword]*len(reviews), review_texts, visit_dates_texts)))

            except Exception as e:
                print(f"4번째 탭 선택 중 에러 발생: {str(e)}")

        except Exception as e:
            print(f"리뷰 탭 클릭 중 에러 발생: {str(e)}")

        # 검색창 기본 프레임으로 전환
        driver.switch_to.default_content()

    except Exception as e:
        print(f"에러 발생: {str(e)}")

    finally:
        # 현재 창과 탭 모두 닫기
        if len(driver.window_handles) > 1:
            for handle in driver.window_handles[1:]:
                driver.switch_to.window(handle)
                driver.close()
            driver.switch_to.window(driver.window_handles[0])

        # 검색창 초기화 (다음 가게로 넘어가기 전에)
        search.clear()

# 데이터프레임 생성
df_result = pd.DataFrame(results, columns=['업체명', '리뷰', '방문일'])

# 데이터프레임 출력
print(df_result)

# 브라우저 닫기
driver.quit()


더보기 버튼이 더 이상 없음: Message: 
Stacktrace:
	GetHandleVerifier [0x0082C1C3+27395]
	(No symbol) [0x007C3DC4]
	(No symbol) [0x006C1B7F]
	(No symbol) [0x00702C65]
	(No symbol) [0x00702D3B]
	(No symbol) [0x0073EC82]
	(No symbol) [0x007239E4]
	(No symbol) [0x0073CB24]
	(No symbol) [0x00723736]
	(No symbol) [0x006F7541]
	(No symbol) [0x006F80BD]
	GetHandleVerifier [0x00AE3A93+2876371]
	GetHandleVerifier [0x00B37F5D+3221661]
	GetHandleVerifier [0x008AD634+556916]
	GetHandleVerifier [0x008B474C+585868]
	(No symbol) [0x007CCE04]
	(No symbol) [0x007C9818]
	(No symbol) [0x007C99B7]
	(No symbol) [0x007BBF0E]
	BaseThreadInitThunk [0x76F5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778180CE+286]
	RtlGetAppContainerNamedObjectPath [0x7781809E+238]

더보기 버튼이 더 이상 없음: Message: 
Stacktrace:
	GetHandleVerifier [0x0082C1C3+27395]
	(No symbol) [0x007C3DC4]
	(No symbol) [0x006C1B7F]
	(No symbol) [0x00702C65]
	(No symbol) [0x00702D3B]
	(No symbol) [0x0073EC82]
	(No symbol) [0x007239E4]
	(No symbol) [0x0073CB24]


In [24]:
df_result

,업체명,리뷰,방문일
0,장수골 충주시,"토종닭누룽지백숙 먹었는데 토종닭 크고 살도 쫀득쫀 득 맛있습니다. 잡내, 비린내 없...",이전 페이지
1,장수골 충주시,장소가 넓어서 매번 가족모임을 하게 되는데 이곳은 골프장 손님들이 대부분이더라고요 ...,이미지 갯수
2,장수골 충주시,해천탕에 해산물도 다양하고 양도 많습니다 !! 닭은 부드럽고 문어도 쫄깃하니 맛있었...,별점
3,장수골 충주시,밑반찬이 맛있고 특히 닭백숙 다먹고 나오는 \n누룽지 닭백숙이 정말 맛있어서 가족모...,
4,장수골 충주시,정말 맛있게먹었습니다! 맛집이라고 예약해서 간 집입니다^^,
...,...,...,...
34265,리얼케익 안양범계점 안양시,육아동지랑 육아 중에 잠시 쉬러 나왔어요~~ 베리요 거트&가나슈 케이크 사먹었는데 ...,반응 남기기
34266,다정김밥 안양시,"평일오전 7시30분~9시30분까지 카드,페이가능 매주 화요일은 쉽니다",이전 페이지
34267,다정김밥 안양시,신메뉴 국물떡볶이 4500원 국물라볶이 6000원 엄 청 푸짐함 라면사리 한개 다 ...,이미지 갯수
34268,다정김밥 안양시,"신동진 쌀. 흑미, 완도 김. 저온 압착 참기름을 사 용합니다. 속이 편한 김밥이라...",


In [33]:
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

In [34]:
def remove_illegal_characters(text):
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub('', text)
    return text

In [35]:
df_result = df_result.applymap(remove_illegal_characters)

# 데이터프레임을 엑셀 파일로 저장
df_result.to_excel('업체정보_리뷰.xlsx', index=False, engine='openpyxl')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7180\2807749165.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_result = df_result.applymap(remove_illegal_characters)


In [28]:
df_result.to_csv('업체정보_리뷰.csv', index=False)